In [3]:
%pwd

'e:\\medibot\\Medical-Chatbot-Generative-AI\\research'

In [4]:
import os
os.chdir('E:\medibot\Medical-Chatbot-Generative-AI')

In [5]:
%pwd

'E:\\medibot\\Medical-Chatbot-Generative-AI'

In [43]:
from langchain_community.document_loaders import PyMuPDFLoader, DirectoryLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [ ]:
def load_pdf_file(data):
    loader = DirectoryLoader(
        data,
        glob="*.pdf",
        loader_cls=PyMuPDFLoader,  # <-- The ONLY change is here
        show_progress=True
    )

    documents = loader.load()

    return documents

100%|██████████| 1/1 [00:15<00:00, 15.60s/it]


In [ ]:
extracted_data = load_pdf_file(data='Data/') 

In [ ]:
def text_split(data):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=500,
        chunk_overlap=20)
    text_chunks = text_splitter.split_documents(extracted_data)
    return text_chunks

In [16]:
text_chunks = text_split(data=extracted_data)
print("Length of text chunks", len(text_chunks))

NameError: name 'text_split' is not defined

In [12]:
from langchain_community.embeddings import HuggingFaceEmbeddings

In [13]:
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(
        model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [14]:
embeddings = download_hugging_face_embeddings()

C:\Users\POOJA\AppData\Local\Temp\ipykernel_25040\4178082895.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(


In [15]:
query_result = embeddings.embed_query("hello world")
print('Length', len(query_result))

Length 384


In [16]:
query_result

[-0.03447727486491203,
 0.03102312609553337,
 0.006734980270266533,
 0.026108933612704277,
 -0.03936205804347992,
 -0.16030246019363403,
 0.06692394614219666,
 -0.006441438104957342,
 -0.047450482845306396,
 0.014758863486349583,
 0.07087534666061401,
 0.05552757531404495,
 0.019193356856703758,
 -0.02625126577913761,
 -0.01010954286903143,
 -0.026940442621707916,
 0.022307462990283966,
 -0.02222665585577488,
 -0.14969263970851898,
 -0.017493024468421936,
 0.007676282897591591,
 0.054352231323719025,
 0.0032544038258492947,
 0.03172588348388672,
 -0.08462139964103699,
 -0.029405992478132248,
 0.051595550030469894,
 0.048124078661203384,
 -0.003314835485070944,
 -0.05827915295958519,
 0.04196925833821297,
 0.022210702300071716,
 0.1281888633966446,
 -0.022338951006531715,
 -0.011656239628791809,
 0.06292837113142014,
 -0.03287634998559952,
 -0.09122604131698608,
 -0.03117534890770912,
 0.052699536085128784,
 0.04703483358025551,
 -0.08420310169458389,
 -0.030056182295084,
 -0.0207448396

In [17]:
from dotenv import load_dotenv
import os
load_dotenv()

True

In [18]:
PINECONE_API_KEY = os.environ.get('PINECONE_API_KEY')
OPENAI_API_KEY = os.environ.get('OPENAI_API_KEY')
GEMINI_API_KEY = os.environ.get('GEMINI_API_KEY')

In [19]:
index_name = 'medibot'

In [27]:
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec

pc = Pinecone(api_key=PINECONE_API_KEY)

pc.create_index(
    name=index_name,
    dimension=384,
    metric="cosine",
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1",
    )
)

PineconeApiException: (409)
Reason: Conflict
HTTP response headers: HTTPHeaderDict({'content-type': 'text/plain; charset=utf-8', 'access-control-allow-origin': '*', 'vary': 'origin,access-control-request-method,access-control-request-headers', 'access-control-expose-headers': '*', 'x-pinecone-api-version': '2025-04', 'x-cloud-trace-context': '86494555eba8840b999ed2dc53fa59c7', 'date': 'Mon, 17 Nov 2025 17:50:23 GMT', 'server': 'Google Frontend', 'Content-Length': '85', 'Via': '1.1 google', 'Alt-Svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000'})
HTTP response body: {"error":{"code":"ALREADY_EXISTS","message":"Resource  already exists"},"status":409}


In [20]:
import os
os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY
os.environ["GEMINI_API_KEY"] = GEMINI_API_KEY

In [21]:
from langchain_pinecone import PineconeVectorStore

In [14]:

docsearch = PineconeVectorStore.from_documents(
    documents=text_chunks,
    index_name=index_name,
    embedding=embeddings
)

NameError: name 'text_chunks' is not defined

In [22]:
# Import the correct class for DENSE vectors
from langchain_pinecone import PineconeVectorStore

# Load the existing index using the standard class
docsearch = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embeddings
)

In [42]:
docsearch

In [23]:
retriever = docsearch.as_retriever(
    search_type="similarity", search_kwargs={"k": 3})

In [24]:
retriever_docs = retriever.invoke('what is Acne?')

In [18]:
retriever_docs

[Document(id='d80cefd5-1642-4045-8e08-aca743a44f3c', metadata={'author': '', 'creationDate': "D:20061016201933+02'00'", 'creationdate': '2006-10-16T20:19:33+02:00', 'creator': 'Adobe Acrobat 6.0', 'file_path': 'Data\\Medical_book.pdf', 'format': 'PDF 1.5', 'keywords': '', 'modDate': "D:20061016220345+02'00'", 'moddate': '2006-10-16T22:03:45+02:00', 'page': 269.0, 'producer': 'PDFlib+PDI 6.0.3 (SunOS)', 'source': 'Data\\Medical_book.pdf', 'subject': '', 'title': '', 'total_pages': 4505.0, 'trapped': ''}, page_content='forms of acne.\nPurpose\nDifferent types of antiacne drugs are used for\ndifferent purposes. For example, lotions, soaps, gels,\nand creams containing benzoyl peroxide or tretinoin\nmay be used to clear up mild to moderately severe\nacne. Isotretinoin (Accutane) is prescribed only for\nvery severe, disfiguring acne.\nAcne is a skin condition that occurs when pores or\nhair follicles become blocked. This blockage allows a\nwaxy material called sebum to collect inside the po

In [37]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    model="models/gemini-2.5-flash",         # YES—this model name works here!
    api_key= GEMINI_API_KEY,
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/",
    temperature= 0.3,
    max_tokens=700
)


In [34]:
from langchain_classic.chains.retrieval import create_retrieval_chain
from langchain_classic.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say you don't know. "
    "Use three sentences maximum and keep the answer concise."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}")
    ]
)

In [35]:
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [42]:
response = rag_chain.invoke({"input": "what is Acromegaly and gigantism?"})
print(response["answer"])

Acromegaly is a disorder caused by the abnormal release of a chemical from the pituitary gland, leading to increased growth in bone and soft tissue, among other disturbances. This chemical is also responsible for gigantism.


In [40]:
# run this to see what your account actually has access to
from openai import OpenAI
import os

# set your Gemini key & base_url (use the same base_url you used when making calls)
os.environ["GEMINI_API_KEY"] = GEMINI_API_KEY   # or set api_key below
client = OpenAI(
    api_key=os.environ.get("GEMINI_API_KEY"),
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/"  # or your base_url
)

models = client.models.list()
for m in models:  # <-- No more ["data"]
    # print name and any useful metadata
    print(m.id)   # <-- Access with .id
    
    # You can inspect all available properties like this:
    print("  keys:", m.model_dump().keys()) 
    
    # optionally print full entry to inspect supported methods
    print(m)
    print("-" * 60)


models/embedding-gecko-001
  keys: dict_keys(['id', 'created', 'object', 'owned_by', 'display_name'])
Model(id='models/embedding-gecko-001', created=None, object='model', owned_by='google', display_name='Embedding Gecko')
------------------------------------------------------------
models/gemini-2.5-pro-preview-03-25
  keys: dict_keys(['id', 'created', 'object', 'owned_by', 'display_name'])
Model(id='models/gemini-2.5-pro-preview-03-25', created=None, object='model', owned_by='google', display_name='Gemini 2.5 Pro Preview 03-25')
------------------------------------------------------------
models/gemini-2.5-flash-preview-05-20
  keys: dict_keys(['id', 'created', 'object', 'owned_by', 'display_name'])
Model(id='models/gemini-2.5-flash-preview-05-20', created=None, object='model', owned_by='google', display_name='Gemini 2.5 Flash Preview 05-20')
------------------------------------------------------------
models/gemini-2.5-flash
  keys: dict_keys(['id', 'created', 'object', 'owned_by', '